<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
from __future__ import print_function
import pandas as pd
import numpy as np
import os
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import ipysheet
import colorsys
import xlsxwriter
import csv
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
import matplotlib.pyplot as plt
import sys

In [ ]:
# import python CWL files

module_path = os.path.abspath(os.path.join('/SSB'))
if module_path not in sys.path:
    sys.path.append(module_path)

from od_matrices import run, collect_groups, plot_od_pixmap, compute_matrix_norms,collect_matrix_entries, generate_hist_plot, generate_bar_plot

module_path = os.path.abspath(os.path.join('../../python/python_src_preprocessing/target_learning'))
if module_path not in sys.path:
    sys.path.append(module_path)

#print(os.listdir('../../python/python_src_preprocessing/target_learning'))
import preprocessing.preprocessing as processing
import plotting.densitymap as densitymap


## OD-Matrices Widget

In [ ]:
# base directory to use
day = "20190905"
base = '/media/veracrypt1/objectstream_{0}/'.format(day)
# 6 7 8 9 10 11 12 13 14 15 16 17 18 19
new_dir = "csv{0}".format(day)
if new_dir not in os.listdir(): 
    os.makedirs(new_dir)

selected_hour = 5
frame = pd.read_csv(os.path.join(base, 'hours', 'hour-{0}.csv'.format(selected_hour)), dtype={'pedestrianId': 'str'})
mapping = pd.read_csv(os.path.join(base, 'mapping_removed.csv'), dtype={'pedestrianId': 'str'})       

In [ ]:
display(run(frame, mapping, prefix='{0}d{1}h'.format(day,selected_hour)))

## Generate OD-Matrices
### OD-Matrix per hour or complete day

#### If files should be saves set write2csv = True!

In [ ]:
od_complete_day = []
com_size = 0

hours = list(range(5,6))

for h in hours:
    frame = pd.read_csv(os.path.join(base, 'hours', 'hour-{0}.csv'.format(h)), dtype={'pedestrianId': 'str'})
    od_complete_day.append(frame)
    com_size += frame.size
    
    #data, source_to_target, name = collect_groups(frame, mapping, data_type = 'ped_count', write2csv=False, prefix='{0}d-{1}h'.format(day,h))

frame_complete = pd.concat(od_complete_day)
groups, od_matrices, origins_str, destinations_str  = collect_groups(frame_complete, mapping, ('H', 1, 1, 0))

In [ ]:

print(origins_str, destinations_str)
od_test = od_matrices[0][2]
print(od_test)

        

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]
plot_od_pixmap(od_test, fig_title="Complete OD-Matrix {0} day".format(day), val_precision=0, file_name='od_matrix_day{0}'.format(day))

In [ ]:
count = np.sum(np.sum(od_matrix[0][2]))
percentages = np.divide(od_matrix[0][2], count)*100

plot_od_pixmap(percentages, fig_title="Percentages OD-Matrix complete {0} day".format(day), val_precision=3, file_name='od_matrix_percentage_day{0}'.format(day))

In [ ]:
flags = np.zeros((10,10))
removed_indexes = []

for i in range(0,10):
    for j in range(0,10):
        if percentages[i,j] < 0.1:
            flags[i,j] = 1
            removed_indexes.append([i-1,j-1])

removed_indexes.append([6,1])
removed_indexes.append([8,7])

plot_od_pixmap(flags, fig_title="Cell average that are < 0.1% in day {0}".format(day), color_type='binary', show_values=False, file_name='od_matrix_bw_day{0}'.format(day))

## Generate Histogram



In [ ]:
h = 8
frame = pd.read_csv(os.path.join(base, 'hours', 'hour-{0}.csv'.format(h)), dtype={'pedestrianId': 'str'})


In [ ]:
data, source_to_target, name = collect_groups(frame, mapping, data_type = 'ped_count', write2csv=False, prefix='{0}d-{1}h'.format(day,h))

In [ ]:
l2 = compute_matrix_norms(data)



In [ ]:
# title, xlabel, filename,
hist_info = ["L2 Matrix", "L2 matrix norm",\
             "hist-L2_{0}_{1}h".format(day,h)]

x_max = np.max(l2[:,1])
generate_hist_plot(h, l2[:,1], x_max=x_max,y_max=100, info=hist_info)

# title, ylabel, filename
bar_info = ["Matrix norm", "L2 matrix norm",\
            "bar-L2_{0}_{1}h".format(day,h)]

generate_bar_plot(h, l2[:,0], l2[:,1],bar_info,show_av=True)

## Plot historgram for each cell value to show variance

In [ ]:
plt.rcParams['figure.figsize'] = [15, 15]
fig, axis = plt.subplots(10, 10, sharex=True, sharey=True)

for origin in range(-1,9):
    for destination in range(-1,9):
        
        entries = collect_matrix_entries(data,origin,destination)
        hist_title = "O:{0} - T:{1}".format(origin,destination)
        hist_xaxis="OD-Matrix entry at ({0},{1})".format(origin,destination)
        hist_file_name = "hist_ped_{0}_{1}_{2}d_{3}h.png".format(origin,destination,day,h)
        hist_info = [hist_title,hist_xaxis,hist_file_name]

        axis[origin+1,destination+1].hist(x=entries[:,2],bins=10,alpha=0.7,rwidth=0.85)
        axis[origin+1,destination+1].set_xlim([0,43])
        axis[origin+1,destination+1].set_ylim([0,100])
        if origin == -1:
            axis[origin+1,destination+1].set_title("{0}".format(destination))

        if destination == -1:
            axis[origin+1,destination+1].set_ylabel("{0}".format(origin))

        #fig.suptitle('Variance of origin to destination distribution \n {0}:00-{1}:00 a.m.'.format(h,h+1), fontsize=18) # or plt.suptitle('Main title')
        #plt.savefig("{0}day-{1}h_hist.png".format(day,h))
        #plt.show()

## Bar plot of OD-Matrix entries

In [ ]:
hours = list(range(5,22))
flags = np.zeros((10,10))
av_matrix = np.zeros((10,10))

for h in hours:
    frame = pd.read_csv(os.path.join(base, 'hours', 'hour-{0}.csv'.format(h)), dtype={'pedestrianId': 'str'})
    data, source_to_target, name = collect_groups(frame, mapping, data_type = 'ped_count', write2csv=False, prefix='{0}d-{1}h'.format(day,h))

    plt.rcParams['figure.figsize'] = [5, 5]
    #fig, axis = plt.subplots(10, 10, sharex=True, sharey=True)

    use_subplots = False

    for origin in range(-1,9):
        for destination in range(-1,9):

            entries = collect_matrix_entries(data,origin,destination)
            plt.rcParams['figure.figsize'] = [7, 7]
            bar_file_name = "bar-ped-{0}-{1}_{2}d_{3}h.png".format(origin,destination,day,h)

            hist_title = " from origin {0} to destination {1} ".format(origin,destination)
            hist_xaxis = "time intervalls [10s]"
            bar_file_name = "hist.png"
            bar_info = [hist_title, hist_xaxis, bar_file_name]

            if use_subplots:

                average = np.average(entries[:,2])
                axis[origin+1,destination+1].hlines(average,0,360,linewidth=2,color='r')
                bar_x = np.arange(len(entries[:,2]))
                axis[origin+1,destination+1].bar(bar_x,entries[:,2], width=0.8)
                plt.xlim([-10, len(entries[:,2])+10])

                if origin == -1:
                    axis[origin+1,target+1].set_title("{0}".format(target))
                if target == -1:
                    axis[origin+1,target+1].set_ylabel("{0}".format(origin))

            else:
                av_matrix[origin+1, destination+1] += np.average(entries[:,2])

                if (av_matrix[origin+1, destination+1] < 1):
                    flags[origin+1,destination+1] += 1
                else:
                    flags[origin+1,destination+1] += 0
                #generate_bar_plot(h,entries[:,0],entries[:,2],bar_info,show_av=True)
                
                
av_matrix = np.divide(av_matrix, 17)

In [ ]:

fig, ax = plt.subplots()
ax.set_xticks(np.arange(10))
ax.set_yticks(np.arange(10))
od_labels = ['-1','0','1','2','3','4','5','6','7','8']
ax.set_xticklabels(od_labels)
ax.set_yticklabels(od_labels)
for i in range(0,10):
    for j in range(0,10):
        text = ax.text(j, i, np.round(av_matrix[i, j],2), ha="center", va="center", color="w")

av_matrix = np.divide(av_matrix,17)
plt.title("Average OD-Matrix {0} day".format(day))
plt.imshow(av_matrix, cmap='viridis')
plt.colorbar()
plt.savefig("average-OD-Matrix-{0}day".format(day),dpi=400)
plt.show()

    
# average counts
fig, ax = plt.subplots()
ax.set_xticks(np.arange(10))
ax.set_yticks(np.arange(10))
od_labels = ['-1','0','1','2','3','4','5','6','7','8']
ax.set_xticklabels(od_labels)
ax.set_yticklabels(od_labels)
for i in range(0,10):
    for j in range(0,10):
        text = ax.text(j, i, flags[i, j], ha="center", va="center", color="r")

        
plt.title("Number of times cell average is < 1 {0} day".format(day))
plt.imshow(flags, cmap='binary')
plt.colorbar()
plt.savefig("average_smaller1_day"+str(day),dpi=400)
plt.show()